In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import urllib.request
from bs4 import BeautifulSoup as bs
import re
from yaml import safe_load

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
all_drinks = pd.read_csv('all_drinks.csv')
hotel_cocktails = pd.read_csv('hotaling_cocktails - Cocktails.csv')
cocktails = pd.read_csv('data_cocktails.csv')
reviews = pd.read_csv('LiqourReviews.csv')
with open('cocktails.yaml', 'r') as f:
    fourth_data = pd.json_normalize(safe_load(f))

In [2]:
def format_all_drinks(old_df):
    new_df = df = pd.DataFrame(columns=['drink_name', 'alcoholic', 'category', 'ingredients', 'quantities', 'instructions', 'picture'])
    ingr_cols = ['strDrink', 'strInstructions', 'strAlcoholic', 'strCategory', 'strDrinkThumb',
                 'strIngredient1', 'strIngredient2', 
                 'strIngredient3', 'strIngredient4', 'strIngredient5', 
                 'strIngredient6', 'strIngredient7', 'strIngredient8', 'strIngredient9', 'strIngredient10', 
                 'strIngredient11', 'strIngredient12','strIngredient13', 'strIngredient14', 'strIngredient15',
                 'strMeasure1', 'strMeasure2', 'strMeasure3', 'strMeasure4', 'strMeasure5','strMeasure6', 
                 'strMeasure7', 'strMeasure8', 'strMeasure9', 'strMeasure10', 'strMeasure11', 'strMeasure12', 
                 'strMeasure13', 'strMeasure14', 'strMeasure15']
    select_df = old_df[ingr_cols]

    for i, j in select_df.iterrows():
        temp_ingredients = []
        temp_quantities = []
        for col in range(5, 15):
            if pd.isna(j[col]) == False:
                temp_ingredients.append(j[col].strip())
                if pd.isna(j[col+15]):
                    temp_quantities.append(" ")
                else:
                    temp_quantities.append(j[col+15].strip('\n').strip(" "))
        new_df.loc[i] = [j['strDrink'], j['strAlcoholic'], j['strCategory'], temp_ingredients, temp_quantities, j['strInstructions'], j['strDrinkThumb']]
    return(new_df)

In [3]:
format_all_drinks(all_drinks)

,drink_name,alcoholic,category,ingredients,quantities,instructions,picture
0,'57 Chevy with a White License Plate,Alcoholic,Cocktail,"[Creme de Cacao, Vodka]","[1 oz white, 1 oz]",1. Fill a rocks glass with ice 2.add white creme de cacao and vodka 3.stir,http://www.thecocktaildb.com/images/media/drink/qyyvtu1468878544.jpg
1,1-900-FUK-MEUP,Alcoholic,Shot,"[Absolut Kurant, Grand Marnier, Chambord raspberry liqueur, Midori melon liqueur, Malibu rum, Amaretto, Cranberry juice, Pineapple juice]","[1/2 oz, 1/4 oz, 1/4 oz, 1/4 oz, 1/4 oz, 1/4 oz, 1/2 oz, 1/4 oz]",Shake ingredients in a mixing tin filled with ice cubes. Strain into a rocks glass.,http://www.thecocktaildb.com/images/media/drink/uxywyw1468877224.jpg
2,110 in the shade,Alcoholic,Beer,"[Lager, Tequila]","[16 oz, 1.5 oz]",Drop shooter in glass. Fill with beer,http://www.thecocktaildb.com/images/media/drink/xxyywq1454511117.jpg
3,151 Florida Bushwacker,Alcoholic,Milk / Float / Shake,"[Malibu rum, Light rum, 151 proof rum, Dark Creme de Cacao, Cointreau, Milk, Coconut liqueur, Vanilla ice-cream]","[1/2 oz, 1/2 oz, 1/2 oz Bacardi, 1 oz, 1 oz, 3 oz, 1 oz, 1 cup]",Combine all ingredients. Blend until smooth. Garnish with chocolate shavings if desired.,http://www.thecocktaildb.com/images/media/drink/rvwrvv1468877323.jpg
4,155 Belmont,Alcoholic,Cocktail,"[Dark rum, Light rum, Vodka, Orange juice]","[1 shot, 2 shots, 1 shot, 1 shot]",Blend with ice. Serve in a wine glass. Garnish with carrot.,http://www.thecocktaildb.com/images/media/drink/yqvvqs1475667388.jpg
...,...,...,...,...,...,...,...
541,Zippy's Revenge,Alcoholic,Cocktail,"[Amaretto, Rum, Kool-Aid]","[2 oz, 2 oz, 4 oz Grape]",Mix Kool-Aid to taste then add Rum and ammaretto. shake well to disolve the sugar in the Kool-Aid... serve cold,http://www.thecocktaildb.com/images/media/drink/1sqm7n1485620312.jpg
542,Zizi Coin-coin,Alcoholic,Punch / Party Drink,"[Cointreau, Lemon juice, Ice, Lemon]","[5 cl, 2 cl, cubes, or lime]","Pour 5cl of Cointreau on ice, add 2cl of fresh lemon (or lime) juice, stir gently, and finally add slices of lemon/lime in glass.",http://www.thecocktaildb.com/images/media/drink/0fbo2t1485620752.jpg
543,Zoksel,Alcoholic,Soft Drink / Soda,"[Beer, Root beer, Lemonade, Coca-Cola, 7-Up, Creme de Cassis, Lemon]","[, , , slice, , , ]","No specific mixinginstructions, just poor every ingredient in one glass. The lemon goes with it.",http://www.thecocktaildb.com/images/media/drink/ft8ed01485620930.jpg
544,Zorbatini,Alcoholic,Cocktail,"[Vodka, Ouzo]","[1 1/4 oz Stoli, 1/4 oz]",Prepare like a Martini. Garnish with a green olive.,http://www.thecocktaildb.com/images/media/drink/wtkqgb1485621155.jpg


In [4]:
def format_cocktails(old_df):
    new_df = df = pd.DataFrame(columns=['drink_name', 'taste', 'category', 'ingredients', 'quantities', 'instructions'])
    
    temp_ingredients = []
    temp_quantities = []
    prev_name = None
    current_pos = 0
    for i, j in old_df.iterrows():
        current_name = j['strDrink']
        if (current_name == prev_name or prev_name == None) == False:
            new_df.loc[current_pos] = [prev_name, j['Basic_taste'], j['strCategory'], temp_ingredients, temp_quantities, j['strInstructions']]
            current_pos += 1
            temp_ingredients = []
            temp_quantities = []
        temp_ingredients.append(j['strIngredients'].strip())
        if pd.isna(j['strMeasures']): temp_quantities.append(" ")
        else: temp_quantities.append(j['strMeasures'].strip())
        prev_name = current_name
        
    return(new_df)

In [5]:
format_cocktails(cocktails)

,drink_name,taste,category,ingredients,quantities,instructions
0,'57 Chevy with a White License Plate,NaN,Shot,"[Creme De Cacao White, Vodka]","[1 oz white, 1 oz]",Shake ingredients in a mixing tin filled with ice cubes. Strain into a rocks glass.
1,1-900-FUK-MEUP,NaN,Beer,"[Grand Marnier, Midori Melon Liqueur, Malibu Rum, Amaretto, Cranberry Juice, Pineapple Juice, Chambord Raspberry Liqueur, Absolut Kurant]","[1/4 oz, 1/4 oz, 1/4 oz, 1/4 oz, 1/2 oz, 1/4 oz, 1/4 oz, 1/2 oz]",Drop shooter in glass. Fill with beer
2,110 in the shade,NaN,Milk / Float / Shake,"[Tequila, Lager]","[1.5 oz, 16 oz]",Combine all ingredients. Blend until smooth. Garnish with chocolate shavings if desired.
3,151 Florida Bushwacker,NaN,Cocktail,"[Dark Creme De Cacao, Coconut Liqueur, Light Rum, Milk, Cointreau, Malibu Rum, Vanilla Ice-Cream, 151 Proof Rum]","[1 oz, 1 oz, 1/2 oz, 3 oz, 1 oz, 1/2 oz, 1 cup, 1/2 oz bacardi]",Blend with ice. Serve in a wine glass. Garnish with carrot.
4,155 Belmont,NaN,Shot,"[Light Rum, Orange Juice, Vodka, Dark Rum]","[2 shots, 1 shot, 1 shot, 1 shot]","Add over ice,shake and pour."
...,...,...,...,...,...,...
467,Zinger,NaN,Shot,"[Surge, Peachtree Schnapps]","[4 shots, 4 shots]","Fill glass with rocks, add straw before putting in liquor. Then add the ingredients in order, trying to keep layered as much as possible (i.e. Chambord on bottom, then Vodka, Then soda on top)."
468,Zipperhead,NaN,Cocktail,"[Chambord Raspberry Liqueur, Soda Water, Vodka]","[1 shot, fill with, 1 shot]",Mix Kool-Aid to taste then add Rum and ammaretto. shake well to disolve the sugar in the Kool-Aid... serve cold
469,Zippy's Revenge,NaN,Punch / Party Drink,"[Rum, Kool-Aid Grape, Amaretto]","[2 oz, 4 oz grape, 2 oz]","Pour 5cl of Cointreau on ice, add 2cl of fresh lemon (or lime) juice, stir gently, and finally add slices of lemon/lime in glass."
470,Zizi Coin-coin,NaN,Cocktail,"[Cointreau, Lemon Juice, Lemon]","[5 cl, 2 cl, or lime]",Prepare like a Martini. Garnish with a green olive.


In [6]:
def format_hotel_cocktails(old_df):
    new_df = df = pd.DataFrame(columns=['drink_name', 'ingredients', 'quantities', 'instructions'])
    
    measures = ['oz', 'dash', 'mist', 'bsp', 'drops', 'top', 'count', 'spoon', 'whole', 'ml', 'float', 
                'tsp', 'c', 'bottle', 'splash', 'soaked', 'part']
    
    for i, j in old_df.iterrows():
        lst = j['Ingredients'].split(',')
        temp_ingredients = []
        temp_quantities = []
        for element in lst:
            current = element.strip()
            spliting_pos = -1
            measure_pos = 0
            pos = -1
            while pos == -1 and measure_pos < len(measures):
                pos = current.find(measures[measure_pos])
                measure_pos += 1
            if measure_pos == len(measures): pos = current.find(" ")
            else: pos += len(measures[measure_pos - 1])
            temp_ingredients.append(current[pos + 1:].strip())
            temp_quantities.append(current[:pos].strip())
        instructions = j['Preparation']
        if pd.isna(instructions) == False: instructions = instructions.replace('\n', ". ")
            
        new_df.loc[i] = [j['Cocktail Name'], temp_ingredients, temp_quantities, instructions]
        
    return(new_df)

In [7]:
fourth_data

,name,timing,taste,ingredients,preparation,source
0,Americano,Pre-dinner,Bitter sweet,"[{'ingredient': 'Campari', 'amount': 3, 'unit': 'cl'}, {'ingredient': 'Sweet Vermouth', 'amount': 3, 'unit': 'cl'}, {'ingredient': 'Soda water', 'amount': 1, 'unit': 'splash'}]",Stirred,NaN
1,Daiquiri,Pre-dinner,Fresh,"[{'ingredient': 'Light Rum', 'amount': 4.5, 'unit': 'cl'}, {'ingredient': 'Lime juice', 'amount': 2.5, 'unit': 'cl'}, {'ingredient': 'Simple syrup', 'amount': 1.5, 'unit': 'cl'}]",Shaken,NaN
2,Hemingway Daiquiri,Pre-dinner,Fresh,"[{'ingredient': 'Light Rum', 'amount': 6, 'unit': 'cl'}, {'ingredient': 'Lime juice', 'amount': 2.5, 'unit': 'cl'}, {'ingredient': 'Maraschino', 'amount': 1.5, 'unit': 'cl'}, {'ingredient': 'Grapefruit juice', 'amount': 1.5, 'unit': 'cl'}]",Shaken,NaN
3,Whiskey Sour,Pre-dinner,Sour,"[{'ingredient': 'Bourbon Whiskey', 'amount': 4.5, 'unit': 'cl'}, {'ingredient': 'Lemon juice', 'amount': 3, 'unit': 'cl'}, {'ingredient': 'Simple syrup', 'amount': 1.5, 'unit': 'cl'}, {'ingredient': 'Egg white', 'amount': 1, 'unit': 'dash'}]",Shaken,NaN
4,Napa Vally Sour,Pre-dinner,Sour,"[{'ingredient': 'Rye Whiskey', 'amount': 6, 'unit': 'cl'}, {'ingredient': 'Lemon juice', 'amount': 2, 'unit': 'cl'}, {'ingredient': 'Red wine syrup', 'amount': 2, 'unit': 'cl'}, {'ingredient': 'Egg white', 'amount': 1, 'unit': 'dash'}]",Shaken,NaN
...,...,...,...,...,...,...
67,Adriatique,All day,Fresh,"[{'ingredient': 'Amaro Montenegro', 'amount': 3, 'unit': 'cl'}, {'ingredient': 'Orange juice', 'amount': 3, 'unit': 'cl'}, {'ingredient': 'Aperol', 'amount': 1.5, 'unit': 'cl'}]",Shaken,NaN
68,Twentieth Century,All day,Fresh,"[{'ingredient': 'Gin', 'amount': 4.5, 'unit': 'cl'}, {'ingredient': 'Lillet Blanc', 'amount': 2, 'unit': 'cl'}, {'ingredient': 'Light crème de cacao', 'amount': 1.5, 'unit': 'cl'}, {'ingredient': 'Lemon juice', 'amount': 2, 'unit': 'cl'}]",Shaken,NaN
69,Irish coffee,After dinner,Boozy,"[{'ingredient': 'Irish Whiskey', 'amount': 4, 'unit': 'cl'}, {'ingredient': 'Coffee', 'amount': 9, 'unit': 'cl'}, {'ingredient': 'Fresh cream', 'amount': 3, 'unit': 'cl'}]",Stirred,IBA
70,Margarita,All day,Fresh,"[{'ingredient': 'Tequila', 'amount': 5, 'unit': 'cl'}, {'ingredient': 'Triple Sec', 'amount': 2, 'unit': 'cl'}, {'ingredient': 'Lime juice', 'amount': 1.5, 'unit': 'cl'}]",Shaken,IBA


In [8]:
fourth_data.loc[0]['ingredients'][0]

{'ingredient': 'Campari', 'amount': 3, 'unit': 'cl'}

In [9]:
def format_fourth(old_df):
    new_df = df = pd.DataFrame(columns=['drink_name', 'taste', 'ingredients', 'quantities', 'instructions'])
    for i, j in old_df.iterrows():
        temp_quantities = []
        temp_ingredients = []
        for key in j['ingredients']:
            temp_ingredients.append(key['ingredient'])
            temp_quantities.append(str(key['amount']) + " " + str(key['unit']))
        if j['preparation'] == 'Shaken': instructions = '1. Add ingredients to cocktail shaker. 2. Fill shaker with ice and mix ingredients. 3. Strain cocktail into a chilled glass.'
        else: instructions = 'Combine ingredients together and serve'
        new_df.loc[i] = [j['name'], j['taste'], temp_ingredients, temp_quantities, instructions]
    return new_df

In [10]:
format_fourth(fourth_data)

,drink_name,taste,ingredients,quantities,instructions
0,Americano,Bitter sweet,"[Campari, Sweet Vermouth, Soda water]","[3 cl, 3 cl, 1 splash]",Combine ingredients together and serve
1,Daiquiri,Fresh,"[Light Rum, Lime juice, Simple syrup]","[4.5 cl, 2.5 cl, 1.5 cl]",1. Add ingredients to cocktail shaker. 2. Fill shaker with ice and mix ingredients. 3. Strain cocktail into a chilled glass.
2,Hemingway Daiquiri,Fresh,"[Light Rum, Lime juice, Maraschino, Grapefruit juice]","[6 cl, 2.5 cl, 1.5 cl, 1.5 cl]",1. Add ingredients to cocktail shaker. 2. Fill shaker with ice and mix ingredients. 3. Strain cocktail into a chilled glass.
3,Whiskey Sour,Sour,"[Bourbon Whiskey, Lemon juice, Simple syrup, Egg white]","[4.5 cl, 3 cl, 1.5 cl, 1 dash]",1. Add ingredients to cocktail shaker. 2. Fill shaker with ice and mix ingredients. 3. Strain cocktail into a chilled glass.
4,Napa Vally Sour,Sour,"[Rye Whiskey, Lemon juice, Red wine syrup, Egg white]","[6 cl, 2 cl, 2 cl, 1 dash]",1. Add ingredients to cocktail shaker. 2. Fill shaker with ice and mix ingredients. 3. Strain cocktail into a chilled glass.
...,...,...,...,...,...
67,Adriatique,Fresh,"[Amaro Montenegro, Orange juice, Aperol]","[3 cl, 3 cl, 1.5 cl]",1. Add ingredients to cocktail shaker. 2. Fill shaker with ice and mix ingredients. 3. Strain cocktail into a chilled glass.
68,Twentieth Century,Fresh,"[Gin, Lillet Blanc, Light crème de cacao, Lemon juice]","[4.5 cl, 2 cl, 1.5 cl, 2 cl]",1. Add ingredients to cocktail shaker. 2. Fill shaker with ice and mix ingredients. 3. Strain cocktail into a chilled glass.
69,Irish coffee,Boozy,"[Irish Whiskey, Coffee, Fresh cream]","[4 cl, 9 cl, 3 cl]",Combine ingredients together and serve
70,Margarita,Fresh,"[Tequila, Triple Sec, Lime juice]","[5 cl, 2 cl, 1.5 cl]",1. Add ingredients to cocktail shaker. 2. Fill shaker with ice and mix ingredients. 3. Strain cocktail into a chilled glass.


In [11]:
formated_all_drinks = format_all_drinks(all_drinks)
formated_cocktails = format_cocktails(cocktails)
formated_hotel_cocktails = format_hotel_cocktails(hotel_cocktails)
fourth_data = format_fourth(fourth_data)

In [12]:
merged_data = formated_all_drinks.append(formated_cocktails)
merged_data = merged_data.append(formated_hotel_cocktails)
merged_data = merged_data.append(fourth_data)
merged_data['drink_name'] = merged_data['drink_name'].apply(lambda x: x.strip())
def make_lower_list(lst):
    return [x.lower() for x in lst]
def make_lower_val(val):
    if pd.isna(val) == False:
        val = val.lower()
        val = val.replace('/', ',')
        return val
    else:
        return val
merged_data['ingredients'] = merged_data['ingredients'].apply(make_lower_list)
merged_data['quantities'] = merged_data['quantities'].apply(make_lower_list)
merged_data['category'] = merged_data['category'].apply(make_lower_val)
merged_data['taste'] = merged_data['taste'].apply(make_lower_val)

In [13]:
merged_data

,drink_name,alcoholic,category,ingredients,quantities,instructions,picture,taste
0,'57 Chevy with a White License Plate,Alcoholic,cocktail,"[creme de cacao, vodka]","[1 oz white, 1 oz]",1. Fill a rocks glass with ice 2.add white creme de cacao and vodka 3.stir,http://www.thecocktaildb.com/images/media/drink/qyyvtu1468878544.jpg,NaN
1,1-900-FUK-MEUP,Alcoholic,shot,"[absolut kurant, grand marnier, chambord raspberry liqueur, midori melon liqueur, malibu rum, amaretto, cranberry juice, pineapple juice]","[1/2 oz, 1/4 oz, 1/4 oz, 1/4 oz, 1/4 oz, 1/4 oz, 1/2 oz, 1/4 oz]",Shake ingredients in a mixing tin filled with ice cubes. Strain into a rocks glass.,http://www.thecocktaildb.com/images/media/drink/uxywyw1468877224.jpg,NaN
2,110 in the shade,Alcoholic,beer,"[lager, tequila]","[16 oz, 1.5 oz]",Drop shooter in glass. Fill with beer,http://www.thecocktaildb.com/images/media/drink/xxyywq1454511117.jpg,NaN
3,151 Florida Bushwacker,Alcoholic,"milk , float , shake","[malibu rum, light rum, 151 proof rum, dark creme de cacao, cointreau, milk, coconut liqueur, vanilla ice-cream]","[1/2 oz, 1/2 oz, 1/2 oz bacardi, 1 oz, 1 oz, 3 oz, 1 oz, 1 cup]",Combine all ingredients. Blend until smooth. Garnish with chocolate shavings if desired.,http://www.thecocktaildb.com/images/media/drink/rvwrvv1468877323.jpg,NaN
4,155 Belmont,Alcoholic,cocktail,"[dark rum, light rum, vodka, orange juice]","[1 shot, 2 shots, 1 shot, 1 shot]",Blend with ice. Serve in a wine glass. Garnish with carrot.,http://www.thecocktaildb.com/images/media/drink/yqvvqs1475667388.jpg,NaN
...,...,...,...,...,...,...,...,...
67,Adriatique,NaN,NaN,"[amaro montenegro, orange juice, aperol]","[3 cl, 3 cl, 1.5 cl]",1. Add ingredients to cocktail shaker. 2. Fill shaker with ice and mix ingredients. 3. Strain cocktail into a chilled glass.,NaN,fresh
68,Twentieth Century,NaN,NaN,"[gin, lillet blanc, light crème de cacao, lemon juice]","[4.5 cl, 2 cl, 1.5 cl, 2 cl]",1. Add ingredients to cocktail shaker. 2. Fill shaker with ice and mix ingredients. 3. Strain cocktail into a chilled glass.,NaN,fresh
69,Irish coffee,NaN,NaN,"[irish whiskey, coffee, fresh cream]","[4 cl, 9 cl, 3 cl]",Combine ingredients together and serve,NaN,boozy
70,Margarita,NaN,NaN,"[tequila, triple sec, lime juice]","[5 cl, 2 cl, 1.5 cl]",1. Add ingredients to cocktail shaker. 2. Fill shaker with ice and mix ingredients. 3. Strain cocktail into a chilled glass.,NaN,fresh


In [14]:
agg_functions = {'drink_name': 'first', 'category': lambda x: ', '.join(x.dropna()), 'ingredients': 'first', 'quantities': 'first', 
                 'instructions': 'first', 'picture': 'first', 'taste': 'first'}
agg_data = merged_data.groupby(merged_data['drink_name'].str.lower(), as_index = False).aggregate(agg_functions)
agg_data

,drink_name,category,ingredients,quantities,instructions,picture,taste
0,#106 Cocktail,,"[chinaco blanco tequila, luxardo bitter bianco, pear syrup*, lemon juice, grey sea salt]","[1 oz, 1 oz, .5 oz, .5 oz, pinc]",Shake and double strain into a coupe. Garnish with sage.. . *Pear Syrup:. -- Pear. -- Sage. -- White Pepper. -- Star Anise,None,None
1,'57 Chevy with a White License Plate,"cocktail, shot","[creme de cacao, vodka]","[1 oz white, 1 oz]",1. Fill a rocks glass with ice 2.add white creme de cacao and vodka 3.stir,http://www.thecocktaildb.com/images/media/drink/qyyvtu1468878544.jpg,None
2,1-900-FUK-MEUP,"shot, beer","[absolut kurant, grand marnier, chambord raspberry liqueur, midori melon liqueur, malibu rum, amaretto, cranberry juice, pineapple juice]","[1/2 oz, 1/4 oz, 1/4 oz, 1/4 oz, 1/4 oz, 1/4 oz, 1/2 oz, 1/4 oz]",Shake ingredients in a mixing tin filled with ice cubes. Strain into a rocks glass.,http://www.thecocktaildb.com/images/media/drink/uxywyw1468877224.jpg,None
3,10 to 7,,"[no. 209 gin, luxardo maraschino liqueur, st. germain, lemon juice]","[1 oz, 1 oz, 1 oz, 1 oz]",Shake and serve up in a martini glass with lemon peel rind.,None,None
4,110 in the shade,"beer, milk , float , shake","[lager, tequila]","[16 oz, 1.5 oz]",Drop shooter in glass. Fill with beer,http://www.thecocktaildb.com/images/media/drink/xxyywq1454511117.jpg,None
...,...,...,...,...,...,...,...
1238,Zizi Coin-coin,"punch , party drink, cocktail","[cointreau, lemon juice, ice, lemon]","[5 cl, 2 cl, cubes, or lime]","Pour 5cl of Cointreau on ice, add 2cl of fresh lemon (or lime) juice, stir gently, and finally add slices of lemon/lime in glass.",http://www.thecocktaildb.com/images/media/drink/0fbo2t1485620752.jpg,None
1239,Zoksel,"soft drink , soda","[beer, root beer, lemonade, coca-cola, 7-up, creme de cassis, lemon]","[, , , slice, , , ]","No specific mixinginstructions, just poor every ingredient in one glass. The lemon goes with it.",http://www.thecocktaildb.com/images/media/drink/ft8ed01485620930.jpg,None
1240,Zorbatini,"cocktail, coffee , tea","[vodka, ouzo]","[1 1/4 oz stoli, 1/4 oz]",Prepare like a Martini. Garnish with a green olive.,http://www.thecocktaildb.com/images/media/drink/wtkqgb1485621155.jpg,None
1241,Zorro,"coffee , tea","[sambuca, bailey's irish cream, white creme de menthe]","[2 cl, 2 cl, 2 cl]",add all and pour black coffee and add whipped cream on top.,http://www.thecocktaildb.com/images/media/drink/kvvd4z1485621283.jpg,None


In [15]:
def separate_ingredients(row):
    new_ingr = ['ingredients1', 'ingredients2', 'ingredients3', 'ingredients4', 'ingredients5', 'ingredients6',
                'ingredients7', 'ingredients8', 'ingredients9', 'ingredients10', 'ingredients11', 'ingredients12']
    new_quant = ['quantity1', 'quantity2', 'quantity3', 'quantity4', 'quantity5', 'quantity6',
                'quantity7', 'quantity8', 'quantity9', 'quantity10', 'quantity11', 'quantity12']

    for pos in range(len(new_ingr)):
        if len(row['ingredients']) > pos:
            row[new_ingr[pos]] = row['ingredients'][pos]
            row[new_quant[pos]] = row['quantities'][pos]
        else:
            row[new_ingr[pos]] = ""
            row[new_quant[pos]] = ""
    return row

In [22]:
new_data = agg_data.apply(separate_ingredients, axis = 1)

In [23]:
new_data = new_data.drop(columns = ['ingredients', 'quantities'])
new_data = new_data[pd.isna(new_data['ingredients1']) == False]
new_data

,drink_name,category,instructions,picture,taste,ingredients1,quantity1,ingredients2,quantity2,ingredients3,quantity3,ingredients4,quantity4,ingredients5,quantity5,ingredients6,quantity6,ingredients7,quantity7,ingredients8,quantity8,ingredients9,quantity9,ingredients10,quantity10,ingredients11,quantity11,ingredients12,quantity12
0,#106 Cocktail,,Shake and double strain into a coupe. Garnish with sage.. . *Pear Syrup:. -- Pear. -- Sage. -- White Pepper. -- Star Anise,None,None,chinaco blanco tequila,1 oz,luxardo bitter bianco,1 oz,pear syrup*,.5 oz,lemon juice,.5 oz,grey sea salt,pinc,,,,,,,,,,,,,,
1,'57 Chevy with a White License Plate,"cocktail, shot",1. Fill a rocks glass with ice 2.add white creme de cacao and vodka 3.stir,http://www.thecocktaildb.com/images/media/drink/qyyvtu1468878544.jpg,None,creme de cacao,1 oz white,vodka,1 oz,,,,,,,,,,,,,,,,,,,,
2,1-900-FUK-MEUP,"shot, beer",Shake ingredients in a mixing tin filled with ice cubes. Strain into a rocks glass.,http://www.thecocktaildb.com/images/media/drink/uxywyw1468877224.jpg,None,absolut kurant,1/2 oz,grand marnier,1/4 oz,chambord raspberry liqueur,1/4 oz,midori melon liqueur,1/4 oz,malibu rum,1/4 oz,amaretto,1/4 oz,cranberry juice,1/2 oz,pineapple juice,1/4 oz,,,,,,,,
3,10 to 7,,Shake and serve up in a martini glass with lemon peel rind.,None,None,no. 209 gin,1 oz,luxardo maraschino liqueur,1 oz,st. germain,1 oz,lemon juice,1 oz,,,,,,,,,,,,,,,,
4,110 in the shade,"beer, milk , float , shake",Drop shooter in glass. Fill with beer,http://www.thecocktaildb.com/images/media/drink/xxyywq1454511117.jpg,None,lager,16 oz,tequila,1.5 oz,,,,,,,,,,,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1238,Zizi Coin-coin,"punch , party drink, cocktail","Pour 5cl of Cointreau on ice, add 2cl of fresh lemon (or lime) juice, stir gently, and finally add slices of lemon/lime in glass.",http://www.thecocktaildb.com/images/media/drink/0fbo2t1485620752.jpg,None,cointreau,5 cl,lemon juice,2 cl,ice,cubes,lemon,or lime,,,,,,,,,,,,,,,,
1239,Zoksel,"soft drink , soda","No specific mixinginstructions, just poor every ingredient in one glass. The lemon goes with it.",http://www.thecocktaildb.com/images/media/drink/ft8ed01485620930.jpg,None,beer,,root beer,,lemonade,,coca-cola,slice,7-up,,creme de cassis,,lemon,,,,,,,,,,,
1240,Zorbatini,"cocktail, coffee , tea",Prepare like a Martini. Garnish with a green olive.,http://www.thecocktaildb.com/images/media/drink/wtkqgb1485621155.jpg,None,vodka,1 1/4 oz stoli,ouzo,1/4 oz,,,,,,,,,,,,,,,,,,,,
1241,Zorro,"coffee , tea",add all and pour black coffee and add whipped cream on top.,http://www.thecocktaildb.com/images/media/drink/kvvd4z1485621283.jpg,None,sambuca,2 cl,bailey's irish cream,2 cl,white creme de menthe,2 cl,,,,,,,,,,,,,,,,,,


In [24]:
new_data.to_csv('merged_data.csv')